<a href="https://www.kaggle.com/code/mdsahadathossentanim/malaria?scriptVersionId=259203723" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import time
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Add, DepthwiseConv2D, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Set style for plots
plt.style.use('ggplot')

In [ ]:
# Define dataset paths
parasitized_path = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized"
uninfected_path = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/Uninfected"

# Check if paths exist
print("Parasitized path exists:", os.path.exists(parasitized_path))
print("Uninfected path exists:", os.path.exists(uninfected_path))

# Count number of images in each directory
num_parasitized = len(os.listdir(parasitized_path))
num_uninfected = len(os.listdir(uninfected_path))
print(f"Number of parasitized images: {num_parasitized}")
print(f"Number of uninfected images: {num_uninfected}")

In [ ]:
# Load images and create labels
def load_images(path, label, img_size=(128, 128)):
    images = []
    labels = []
    for filename in os.listdir(path):
        if filename.endswith('.png'):
            img = cv2.imread(os.path.join(path, filename))
            if img is not None:
                img = cv2.resize(img, img_size)
                images.append(img)
                labels.append(label)
    return images, labels

# Load parasitized and uninfected images
parasitized_images, parasitized_labels = load_images(parasitized_path, 1)
uninfected_images, uninfected_labels = load_images(uninfected_path, 0)

# Combine datasets
X = np.array(parasitized_images + uninfected_images)
y = np.array(parasitized_labels + uninfected_labels)

print(f"Total images: {X.shape[0]}")
print(f"Image shape: {X.shape[1:]}")

In [ ]:
# Visualize some samples
plt.figure(figsize=(12, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(X[i])
    plt.title('Parasitized' if y[i] == 1 else 'Uninfected')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Check class distribution
plt.figure(figsize=(8, 5))
sns.countplot(x=y)
plt.title('Class Distribution')
plt.xticks([0, 1], ['Uninfected', 'Parasitized'])
plt.show()

print(f"Uninfected: {np.sum(y==0)}")
print(f"Parasitized: {np.sum(y==1)}")

In [ ]:
# Preprocess data
# Normalize pixel values
X = X.astype('float32') / 255.0

# Convert labels to categorical
y = to_categorical(y, 2)

# Split the data into training, validation, and test sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, random_state=42, stratify=y_temp)

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create data generators
train_generator = datagen.flow(X_train, y_train, batch_size=32)
val_generator = datagen.flow(X_val, y_val, batch_size=32)

In [ ]:
# Define model architectures

# 1. Custom Tiny CNN
def create_tiny_cnn(input_shape=(128, 128, 3), num_classes=2):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# 2. MobileNetV3Small (lightweight version) - ALTERNATIVE
def create_mobilenetv3(input_shape=(128, 128, 3), num_classes=2):
    base_model = MobileNetV3Small(
        weights='imagenet', 
        include_top=False, 
        input_shape=input_shape,
        alpha=0.75,  # Valid for non-minimalistic
        minimalistic=False  # Use regular model
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Add these imports at the top of your imports section
from tensorflow.keras.layers import Concatenate, AveragePooling2D, Lambda
from tensorflow.keras import backend as K

# Corrected SqueezeNet implementation
def fire_module(x, squeeze, expand):
    # Squeeze
    x = Conv2D(squeeze, (1, 1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    # Expand
    expand1x1 = Conv2D(expand, (1, 1), activation='relu', padding='same')(x)
    expand3x3 = Conv2D(expand, (3, 3), activation='relu', padding='same')(x)
    
    x = Concatenate()([expand1x1, expand3x3])
    return x

def create_squeezenet(input_shape=(128, 128, 3), num_classes=2):
    input = Input(shape=input_shape)
    
    x = Conv2D(16, (3, 3), strides=(2, 2), activation='relu', padding='same')(input)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    x = fire_module(x, squeeze=8, expand=16)
    x = fire_module(x, squeeze=8, expand=16)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    x = fire_module(x, squeeze=16, expand=32)
    x = fire_module(x, squeeze=16, expand=32)
    
    x = Dropout(0.5)(x)
    x = Conv2D(num_classes, (1, 1), activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax')(x)
    
    model = Model(inputs=input, outputs=x)
    return model

# Corrected ShuffleNet implementation
def shuffle_unit(x, groups, channels, stride):
    shortcut = x
    
    # Pointwise group convolution
    x = Conv2D(channels, (1, 1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    # Channel shuffle
    x = Lambda(channel_shuffle, arguments={'groups': groups})(x)
    
    # Depthwise convolution
    x = DepthwiseConv2D((3, 3), strides=stride, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    # Pointwise group convolution
    x = Conv2D(channels, (1, 1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    # Adjust shortcut if needed
    if stride > 1:
        shortcut = AveragePooling2D((stride, stride), strides=stride, padding='same')(shortcut)
    
    # Adjust channel dimensions if they don't match
    if K.int_shape(shortcut)[-1] != channels:
        shortcut = Conv2D(channels, (1, 1), activation='relu', padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    x = Add()([shortcut, x])
    return x

def create_shufflenet(input_shape=(128, 128, 3), num_classes=2, groups=2):
    input = Input(shape=input_shape)
    
    x = Conv2D(24, (3, 3), strides=2, activation='relu', padding='same')(input)  # Increased initial channels
    x = MaxPooling2D((3, 3), strides=2, padding='same')(x)
    
    x = shuffle_unit(x, groups=groups, channels=48, stride=1)
    x = shuffle_unit(x, groups=groups, channels=48, stride=2)
    
    x = shuffle_unit(x, groups=groups, channels=96, stride=1)
    x = shuffle_unit(x, groups=groups, channels=96, stride=2)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=x)
    return model

In [ ]:
# Create models
tiny_cnn = create_tiny_cnn()
mobilenet = create_mobilenetv3()
squeezenet = create_squeezenet()
shufflenet = create_shufflenet()

# Compile models
models = {
    'Tiny CNN': tiny_cnn,
    'MobileNetV3': mobilenet,
    'SqueezeNet': squeezenet,
    'ShuffleNet': shufflenet
}

# Count parameters for each model
for name, model in models.items():
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    print(f"{name} parameters: {model.count_params():,}")

In [ ]:
# Train models and store history
history_dict = {}
train_times = {}

# Training parameters
epochs = 30
batch_size = 32

for name, model in models.items():
    print(f"\nTraining {name}...")
    start_time = time.time()
    
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=len(X_val) // batch_size,
        verbose=1
    )
    
    end_time = time.time()
    train_time = end_time - start_time
    train_times[name] = train_time
    history_dict[name] = history.history
    
    print(f"{name} training completed in {train_time:.2f} seconds")

In [ ]:
# Evaluate models on test set
results = {}

for name, model in models.items():
    print(f"\nEvaluating {name}...")
    
    # Predict on test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    cm = confusion_matrix(y_true, y_pred_classes)
    cr = classification_report(y_true, y_pred_classes, target_names=['Uninfected', 'Parasitized'])
    
    # Store results
    results[name] = {
        'test_accuracy': test_accuracy,
        'test_loss': test_loss,
        'confusion_matrix': cm,
        'classification_report': cr,
        'model_size_params': model.count_params()
    }
    
    print(f"{name} Test Accuracy: {test_accuracy:.4f}")
    print(f"Confusion Matrix:\n{cm}")

In [ ]:
# Compare model performance
comparison = pd.DataFrame({
    'Model': list(results.keys()),
    'Test Accuracy': [results[name]['test_accuracy'] for name in results.keys()],
    'Test Loss': [results[name]['test_loss'] for name in results.keys()],
    'Parameters': [results[name]['model_size_params'] for name in results.keys()],
    'Training Time (s)': [train_times[name] for name in results.keys()]
})

print("Model Comparison:")
print(comparison)

In [ ]:
# Plot training history for each model
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, (name, history) in enumerate(history_dict.items()):
    axes[i].plot(history['accuracy'], label='Training Accuracy')
    axes[i].plot(history['val_accuracy'], label='Validation Accuracy')
    axes[i].set_title(f'{name} - Accuracy')
    axes[i].set_xlabel('Epoch')
    axes[i].set_ylabel('Accuracy')
    axes[i].legend()
    
    # Plot loss on the right side
    ax2 = axes[i].twinx()
    ax2.plot(history['loss'], label='Training Loss', color='red', linestyle='--')
    ax2.plot(history['val_loss'], label='Validation Loss', color='orange', linestyle='--')
    ax2.set_ylabel('Loss')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
colors = ['blue', 'red', 'orange', 'green']  # last one is best model (ShuffleNet)
bars = plt.bar(comparison['Model'], comparison['Test Accuracy'], color=colors)

plt.title('Test Accuracy Comparison of Lightweight CNN Models', pad=20)
plt.ylabel('Accuracy')
plt.ylim(0.4, 1.0)   # fixed so all bars are visible

# Add values on top of bars
for bar, accuracy in zip(bars, comparison['Test Accuracy']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{accuracy:.4f}', ha='center', va='bottom')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Example comparison data (replace with your dataframe values)
comparison = {
    'Model': ["ShuffleNet", "Tiny CNN", "SqueezeNet", "MobileNetV3"],
    'Test Accuracy': [0.9625, 0.9519, 0.9146, 0.5000],
    'Parameters': [54578, 826594, 15730, 611002]
}

import pandas as pd
comparison = pd.DataFrame(comparison)

# Identify the best model (highest accuracy)
best_model_idx = comparison['Test Accuracy'].idxmax()

plt.figure(figsize=(10, 6))

# Plot all models in blue
plt.scatter(comparison['Parameters'], comparison['Test Accuracy'], s=100, c='blue', label="Models")

# Highlight best model with a green star
plt.scatter(comparison['Parameters'][best_model_idx],
            comparison['Test Accuracy'][best_model_idx],
            s=200, c='green', marker='*', edgecolors='black',
            label=f"Best Model: {comparison['Model'][best_model_idx]}")

# Add labels for each point
for i, model in enumerate(comparison['Model']):
    plt.annotate(model,
                 (comparison['Parameters'][i], comparison['Test Accuracy'][i]),
                 xytext=(8, 8), textcoords='offset points',
                 fontsize=9, fontweight="bold")

# Log scale for parameters
plt.xscale('log')
plt.xlabel('Number of Parameters (log scale)', fontsize=12)
plt.ylabel('Test Accuracy', fontsize=12)
plt.title('Model Size vs Accuracy', fontsize=14, fontweight="bold")

# Add baseline reference line at 90% accuracy
plt.axhline(0.90, color='gray', linestyle='--', linewidth=1, label="90% Accuracy Threshold")

# Grid + legend
plt.grid(True, which="both", ls="--", alpha=0.6)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Plot confusion matrices
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, (name, result) in enumerate(results.items()):
    cm = result['confusion_matrix']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                xticklabels=['Uninfected', 'Parasitized'],
                yticklabels=['Uninfected', 'Parasitized'])
    axes[i].set_title(f'{name} Confusion Matrix')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('True')

plt.tight_layout()
plt.show()

In [ ]:
# Display detailed results for each model
for name, result in results.items():
    print(f"\n{name} Results:")
    print("=" * 50)
    print(f"Test Accuracy: {result['test_accuracy']:.4f}")
    print(f"Test Loss: {result['test_loss']:.4f}")
    print(f"Number of Parameters: {result['model_size_params']:,}")
    print(f"Classification Report:\n{result['classification_report']}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Example results dictionary (replace with your real `results`)
results = {
    "ShuffleNet": {
        "test_accuracy": 0.9625,
        "test_loss": 0.1234,
        "model_size_params": 54578,
        "classification_report": {
            "Precision": [0.95, 0.96],
            "Recall": [0.94, 0.97],
            "F1-Score": [0.94, 0.96],
            "Class": ["Negative", "Positive"]
        }
    },
    "Tiny CNN": {
        "test_accuracy": 0.9519,
        "test_loss": 0.1456,
        "model_size_params": 826594,
        "classification_report": {
            "Precision": [0.92, 0.95],
            "Recall": [0.91, 0.96],
            "F1-Score": [0.91, 0.95],
            "Class": ["Negative", "Positive"]
        }
    },
    "SqueezeNet": {
        "test_accuracy": 0.9146,
        "test_loss": 0.2001,
        "model_size_params": 15730,
        "classification_report": {
            "Precision": [0.89, 0.92],
            "Recall": [0.87, 0.91],
            "F1-Score": [0.88, 0.91],
            "Class": ["Negative", "Positive"]
        }
    }
}

# Plot summary charts for each model
for name, result in results.items():
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(f"{name} Results", fontsize=14, fontweight="bold")

    # Left side: Accuracy, Loss, Params
    metrics = {
        "Accuracy": result["test_accuracy"],
        "Loss": result["test_loss"],
        "Params": result["model_size_params"] / 1e3  # show in 'K'
    }
    axs[0].bar(metrics.keys(), metrics.values(), color=["skyblue", "salmon", "orange"])
    axs[0].set_title("Overall Metrics")
    axs[0].set_ylabel("Value (Params in K)")

    # Right side: Classification Report
    df = pd.DataFrame(result["classification_report"])
    df.set_index("Class", inplace=True)
    df.plot(kind="bar", ax=axs[1], colormap="viridis")
    axs[1].set_title("Classification Report")
    axs[1].set_ylim(0, 1.1)

    plt.tight_layout()
    plt.show()


In [ ]:
# Make predictions on sample test images
def predict_sample_images(model, X_sample, y_sample, class_names):
    plt.figure(figsize=(12, 8))
    for i in range(12):
        plt.subplot(3, 4, i+1)
        img = X_sample[i]
        true_label = class_names[np.argmax(y_sample[i])]
        
        # Predict
        prediction = model.predict(np.expand_dims(img, axis=0))
        predicted_label = class_names[np.argmax(prediction)]
        
        # Display image
        plt.imshow(img)
        plt.title(f'True: {true_label}\nPred: {predicted_label}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Select the best model based on accuracy
best_model_name = comparison.loc[comparison['Test Accuracy'].idxmax(), 'Model']
best_model = models[best_model_name]

print(f"Best model: {best_model_name}")
predict_sample_images(best_model, X_test[:12], y_test[:12], ['Uninfected', 'Parasitized'])

In [ ]:
# Save the best model for potential deployment
best_model.save(f'malaria_detection_{best_model_name.lower().replace(" ", "_")}.h5')
print(f"Best model saved as 'malaria_detection_{best_model_name.lower().replace(' ', '_')}.h5'")

# Convert to TensorFlow Lite for mobile deployment (optional)
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
tflite_model = converter.convert()

with open(f'malaria_detection_{best_model_name.lower().replace(" ", "_")}.tflite', 'wb') as f:
    f.write(tflite_model)
print(f"TFLite model saved as 'malaria_detection_{best_model_name.lower().replace(' ', '_')}.tflite'")

In [ ]:
# Final analysis and conclusion
print("FINAL ANALYSIS")
print("=" * 50)
print("Based on the experimental results, the performance of the models is as follows:\n")

# Sort models by accuracy
sorted_comparison = comparison.sort_values('Test Accuracy', ascending=False)

for i, (_, row) in enumerate(sorted_comparison.iterrows()):
    print(f"{i+1}. {row['Model']}:")
    print(f"   Accuracy: {row['Test Accuracy']:.4f}")
    print(f"   Parameters: {row['Parameters']:,}")
    print(f"   Training Time: {row['Training Time (s)']:.2f} seconds\n")

# Determine the best model for mobile deployment
best_mobile_model = sorted_comparison.iloc[0]
print(f"BEST MODEL FOR MOBILE DEPLOYMENT: {best_mobile_model['Model']}")
print(f"Rationale: Achieves the highest accuracy ({best_mobile_model['Test Accuracy']:.4f}) ")
print(f"with reasonable model size ({best_mobile_model['Parameters']:,} parameters) ")
print(f"and training time ({best_mobile_model['Training Time (s)']:.2f} seconds).")

In [ ]:
import matplotlib.pyplot as plt

# Data from your comparison output
models = ["ShuffleNet", "Tiny CNN", "SqueezeNet", "MobileNetV3"]
accuracy = [0.9625, 0.9519, 0.9146, 0.5000]
parameters = [54578, 826594, 15730, 611002]
training_time = [2889.95, 2861.75, 2020.50, 1874.55]

best_model_index = accuracy.index(max(accuracy))

# Create subplots
fig, axs = plt.subplots(3, 1, figsize=(10, 12))
fig.suptitle("FINAL ANALYSIS: Model Comparison", fontsize=16, fontweight="bold")

# Accuracy plot
axs[0].bar(models, accuracy, color="skyblue")
axs[0].set_title("Test Accuracy")
axs[0].set_ylabel("Accuracy")
axs[0].bar(models[best_model_index], accuracy[best_model_index], color="green")  # highlight best model

# Parameters plot (log scale for clarity)
axs[1].bar(models, parameters, color="orange")
axs[1].set_title("Number of Parameters")
axs[1].set_ylabel("Parameters (log scale)")
axs[1].set_yscale("log")
axs[1].bar(models[best_model_index], parameters[best_model_index], color="green")  

# Training time plot
axs[2].bar(models, training_time, color="salmon")
axs[2].set_title("Training Time (seconds)")
axs[2].set_ylabel("Seconds")
axs[2].bar(models[best_model_index], training_time[best_model_index], color="green")  

# Highlight text
plt.figtext(0.5, 0.02,
            f"BEST MODEL FOR MOBILE DEPLOYMENT: {models[best_model_index]} "
            f"(Accuracy: {accuracy[best_model_index]:.4f}, "
            f"Parameters: {parameters[best_model_index]:,}, "
            f"Training Time: {training_time[best_model_index]:.2f}s)",
            ha="center", fontsize=12, bbox={"facecolor":"lightyellow", "alpha":0.5, "pad":5})

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()
